In [1]:
import requests
import random
import os
import pandas as pd
from rdkit import Chem

upload_url=r'http://xundrug.cn:5001/modules/upload0/'

def predict_pka(smi):
    param={"Smiles" : ("tmg", smi)}
    headers={'token':'O05DriqqQLlry9kmpCwms2IJLC0MuLQ7'}
    response=requests.post(url=upload_url, files=param, headers=headers)
    jsonbool=int(response.headers['ifjson'])
    if jsonbool==1:
        res_json=response.json()
        if res_json['status'] == 200:
            pka_datas = res_json['gen_datas']
            return pka_datas
        else:
            raise RuntimeError("Error for pKa prediction")
    else:
        raise RuntimeError("Error for pKa prediction")

In [2]:
#BBB Score
#Gupta, M.; Lee, H. J.; Barden, C. J.; Weaver, D. F. The Blood–Brain Barrier (BBB) Score. J. Med. Chem. 2019, 62 (21), 9824–9836. https://doi.org/10.1021/acs.jmedchem.9b01220.

def pka_bbb(base,acid):
    if len(base) == 0 and len(acid) == 0:
        # If molecule has neither (neutral molecule), 
        # take the pKa that would give the maximal score for pKa descriptor (pKa=0 for MPO, pKa = 8.81 for BBB)
        pka = 8.81
        
    elif len(base) != 0 and max(base) >= 5:
        # If the molecule has basic pKa's ≥ 5, take the most basic pKa
        pka = max(base)
    
    elif len(base) == 0 and len(acid) != 0:
        if max(acid) >= 9.0:
            pka = 9.0
        else:
            pka = max(acid)
    elif len(base) != 0 and max(base) < 5 and len(acid) != 0:
        pka = max(acid)
    return pka

In [3]:
# run 
smi = "CC(=O)Nc1ccc(O)cc1"
data_pka = predict_pka(smi)

In [4]:
# get result
base = data_pka['Base'].values()
acid = data_pka['Acid'].values()

In [5]:
# String to number
base = list(map(float,list(base)))
acid = list(map(float,list(acid)))

In [6]:
print('Base pka',base)
print('Acid pka:',acid)

Base pka []
Acid pka: [13.84, 10.1]


In [7]:
x = pka_bbb(base,acid)
print('BBB pka:', x)

BBB pka: 9.0
